# Parameter tuning for time series forecasting

## Manual

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from aeon.datasets import load_airline
from aeon.utils.plotting import plot_series
from aeon.forecasting.model_selection import temporal_train_test_split
from aeon.forecasting.arima import ARIMA
from aeon.performance_metrics.forecasting import mean_absolute_percentage_error
from aeon.forecasting.model_selection import ExpandingWindowSplitter, SlidingWindowSplitter
from utils import plot_windows, get_windows


### Setting up

In [ ]:
y = load_airline()
plot_series(y)
plt.show()

In [ ]:
y_train, y_test = temporal_train_test_split(y, test_size=36)

In [ ]:
fh=np.arange(len(y_test)) + 1
print(fh)

### ExpandingWindowSplitter

In [ ]:
splitter = ExpandingWindowSplitter(fh=fh, initial_window=50, step_length=1)
train_windows, test_windows = get_windows(y_train, splitter)
plot_windows(y_train, train_windows, test_windows)

In [ ]:
# Manually extract indices
splitter = ExpandingWindowSplitter(fh=fh, initial_window=50, step_length=1)
cvs = splitter.split(y_train)

datas = []
for iTrain, iTest in cvs:
    data = {"train": iTrain, "test": iTest}
    datas.append(data)

df = pd.DataFrame.from_dict(datas)
display(df.head(10))

### SlidingWindowSplitter

In [ ]:
splitter = SlidingWindowSplitter(fh=fh, window_length=50, step_length=1)
train_windows, test_windows = get_windows(y_train, splitter)
plot_windows(y_train, train_windows, test_windows)

In [ ]:
# Manually extract indices
splitter = SlidingWindowSplitter(fh=fh, window_length=50, step_length=1)
cvs = splitter.split(y_train)

datas = []
for iTrain, iTest in cvs:
    data = {"train": iTrain, "test": iTest}
    datas.append(data)

df = pd.DataFrame.from_dict(datas)
display(df.head(10))

### Cross validation (manual)

In [ ]:
forecaster = ARIMA(order=(1, 1, 0), seasonal_order=(1, 1, 0, 12))  
splitter = ExpandingWindowSplitter(fh=fh, initial_window=50, step_length=1)
cvs = splitter.split(y_train)

datas = []
for ix, iy in cvs:
    # Get data
    y_train_cv = y_train.iloc[ix]
    y_test_cv = y.iloc[iy]
    # Train the model
    forecaster.fit(y_train_cv)
    # Predict
    y_pred_cv = forecaster.predict(fh)
    # Evaluate
    mape = mean_absolute_percentage_error(y_test_cv, y_pred_cv)
    # storing the results
    data = {"mape": mape}
    datas.append(data)

df = pd.DataFrame.from_dict(datas)
display(df.describe())


### Parameter tuning

In [ ]:
sOrders = np.arange(8, 15)

datas = []
for sOrder in sOrders:
    
    forecaster = ARIMA(order=(1, 1, 0), seasonal_order=(1, 1, 0, sOrder))
    # Need to recompute the splitter because the iterator is exhausted
    splitter = ExpandingWindowSplitter(fh=fh, initial_window=50, step_length=1) 
    cvs = splitter.split(y_train)
    
    datasTemp = [] 
    for ix, iy in cvs:
        # Get data
        y_train_cv = y_train.iloc[ix]
        y_test_cv = y.iloc[iy]
        # Train the model
        forecaster.fit(y_train_cv)
        # Predict
        y_pred_cv = forecaster.predict(fh)
        # Evaluate
        mape = mean_absolute_percentage_error(y_test_cv, y_pred_cv)
        # storing the results
        data = {"mape": mape}
        datasTemp.append(data)

    dfTemp = pd.DataFrame.from_dict(datasTemp)
    data = {
        "sOrder": sOrder,
        "mapeMean": dfTemp["mape"].mean(),
        "mapeStd": dfTemp["mape"].std()
    }
    datas.append(data)

df = pd.DataFrame.from_dict(datas)
display(df)

In [ ]:
df.plot(x="sOrder", y="mapeMean", yerr="mapeStd")
plt.show()

### Retrain

In [ ]:
sOrder_best = df.sort_values(by="mapeMean").iloc[0]['sOrder']
print(sOrder_best)
forecaster = ARIMA(order=(1, 1, 0), seasonal_order=(1, 1, 0, sOrder_best))
forecaster.fit(y_train)

### Test result

In [ ]:
y_pred = forecaster.predict(fh)
mape = mean_absolute_percentage_error(y_test, y_pred)
print(f"mape: {mape}")